In [11]:
import importlib


from camel.agents import ChatAgent
from camel.messages import BaseMessage


In [17]:
from agents.stroy_arch_writer import create_story_arch_writer  
import agents.stroy_arch_writer
importlib.reload(agents.stroy_arch_writer)
def refine_user_input(max_steps=10) -> str:
    refinement_agent = create_story_arch_writer()

    user_msg = input("Please specify your story.")

    initial_user_str = (
        f"""<task>
                Verify whether the provided story arc includes all necessary elements for a Dungeons & Dragons one-shot narrative.
            </task>
            <criteria>
                <element>
                    <name>Plot Hook</name>
                    <description>A compelling reason to engage the players in the story.</description>
                </element>
                <element>
                    <name>Environment/Setting</name>
                    <description>A defined and immersive location where the story unfolds.</description>
                </element>
                <element>
                    <name>Main Thread/Conflict</name>
                    <description>A clear goal or conflict driving the story.</description>
                </element>
                <element>
                    <name>NPCs</name>
                    <description>Engaging non-player characters that add depth and support the narrative.</description>
                </element>
                <element>
                    <name>Surprise Elements</name>
                    <description>Optional twists or unexpected elements to intrigue players.</description>
                </element>
            </criteria>
            <instructions>
                <rule>Do not include player characters (heroes) in the assessment, as they are played by real individuals.</rule>
                <rule>Do not summarize the story unless explicitly asked.</rule>
                <rule>If all elements are covered and no further questions are needed, respond with "<NO_QUESTION>".</rule>
                <rule>If any element is missing or unclear, ask specific questions to gather more details.</rule>
            </instructions>
            <input>
                <description>Here is the user message:</description>
                <content>{user_msg}</content>
            </input>""")

    initial_user_msg = BaseMessage.make_user_message("User", initial_user_str)
    response = refinement_agent.step(initial_user_msg, )

    for i in range(max_steps):
        if i == max_steps - 1:
            break
        if "<NO_QUESTION>" in response.msg:
            break

        # print_text_animated(response.msg.content)
        print(response.msg.content)

        refined_user_input = input("Please answer the given questions.")
        refined_user_msg = BaseMessage.make_user_message(
            "User",
            "Here is an update one the story we build."
            + "{{" + refined_user_input + "}}")
        response = refinement_agent.step(refined_user_msg)

    result = refinement_agent.step(
        BaseMessage.make_user_message(
            "User",
            "Give a story overview with the information give by the user."
            "Structure the summary in the following blocks: player hook, story summary, important characters."
            "The story summary should give detailed information about necessary story elements."))

    # print_text_animated("Thank you for your input. Here is what i got: " + result.msg.content)
    print("Thank you for your input. Here is what i got: " + result.msg.content)
    return result.msg.content
story = refine_user_input()

## Assess Elements:

### Plot Hook:
- The premise of a warchief falling ill creates an immediate sense of urgency and stakes. The heroes' quest to find a cure serves as a compelling motivation that can engage both the characters and the readers.

### Environment:
- The setting is not described. We need to know where this story takes place (e.g., a fantasy realm, a post-apocalyptic world, etc.) and how the environment influences the quest for the cure.

### Thread:
- There is a central narrative thread involving the heroes' quest to find a cure for the warchief. However, more details about the nature of the illness, the challenges they face, and the significance of the warchief to the world or the heroes would strengthen this aspect.

## Optional Enhancements:

### Surprise Elements:
- There are no surprise elements mentioned in the current description. Adding unexpected twists or obstacles could enhance the intrigue of the story.

### Interesting Characters:
- The heroes are mentioned,

KeyboardInterrupt: Interrupted by user

In [ ]:
structure_msg = BaseMessage.make_assistant_message(
    "Scene creator",
    "You are outlining the story in steps. Each step/encounter drives the story forward or reveal more information."
    "You have three broad encounter options: combat, NPC interaction, and exploration (traps, puzzles, navigation). The goal is to make sure you don't have too much of the same stuff back-to-back."
)
structure_agent = ChatAgent(structure_msg)

firstLastResponse = structure_agent.step(
    BaseMessage.make_user_message(
        "User", 
        "Given the story, define the first and last scene/encounter."
        "Each scene should be described in one sentence and should be labeled with an encounter option."
        "For example: "
        "- Follow the thief through the busteling city alleys (navigation)"
        "- Sorcerous orbs speaking in riddles (puzzle)"
        "- Secretive librarian (NPC interaction)"
        "- Final fight with thief and his friends (combat)"
        "Here is the story: "
        "{{" + story + "}}"
        "Define the first and last scene/encounter in the format of:"
        "Story Start: <first scene>"
        "Story End: <last scene>"))
print(firstLastResponse.msg.content)

rows = firstLastResponse.msg.content.split("\n")


In [ ]:
def single_agent_create_scene():
    response = structure_agent.step(
        BaseMessage.make_user_message(
            "User",
            "You need to find 4 to 8 encounters inbetween the first and the last scene."
            "Each step should drive the story forward and offers clues to the next step"
            "They should be ordered according to how the story unfolds"
            "Do not include the first and the last step which are"
            + firstLastResponse.msg.content +
            "For example: "
            "2. Follow the thief through the busteling city alleys (navigation)"
            "3. Sorcerous orbs speaking in riddles in the library (puzzle)"
            "4. Secretive librarian (NPC interaction)"
        )
    )
    print(response.msg.content)
    return response.msg.content


response_content = single_agent_create_scene()

transition_steps = response_content.split("\n")
all_steps = [rows[0], *transition_steps, rows[1]]
print(all_steps)

In [ ]:
#load example scenes
examples = []
with open("../res/scene-example-1.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-2.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-3.txt", "r") as file:
    examples.append(file.read())
    

In [ ]:
# setup scene writer
scene_msg = BaseMessage.make_assistant_message(
    "Scene writer",
    "You write descriptions for scenes played in dnd. "
    "You are writing for the Dungeon Master(DM), not the players. "
    "Remember to make the information easy and fast for the DM to read."
    "The scene you are writing is reference material, not a novel. The DM will be using this live at the table to run games. Split your information out into bullet points and lists, use bolding, get to the point as fast as you can, and cut anything that doesn't matter to the here-and-now (do not explain the history of each room)."
    "Here are examples of how to do it:"
    ""
    "## Example 1:"
    + examples[0] +
    "## Example 2:"
    + examples[1] +
    "## Example 3:"
    + examples[2] +
    "Don't forget you are writing descriptions for dnd scenes"
)
scene_agent = ChatAgent(scene_msg)

In [ ]:

dm_msg = BaseMessage.make_assistant_message(
    "Dungeon Master",
    "You are the dungeon master runnng the discussed game. You want to create an emerging and interesting experience for the players."
    "You need to know how the story will unfold. Which steps are optional and which one can be run in different order."
)
dm_agent = ChatAgent(dm_msg)


bm_msg = BaseMessage.make_assistant_message(
    "Beast Master",
    "You know everything about how to balance a good fight and which monsters are allowed in an dnd game."
    "Make sure the right terms are used when adressing monsters or enemies."
)
bm_agent = ChatAgent(bm_msg)

In [ ]:
from camel.tasks import Task
from camel.workforce import Workforce


def single_agent_write_scene(prompt):
    scene_response = scene_agent.step(BaseMessage.make_user_message(
        "User",
        prompt
    ))
    return scene_response.msg.content

def workforce_write_scene(prompt):
    workforce = Workforce("Dungeons and Dragons creative team"
                              ## define new_agent_kwargs
                              )
    workforce.add_single_agent_worker(
      "Agent that writes detailed descriptions about scenes",
      worker=scene_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that will use the scene description to run a game of dnd. Acts as a validator for the scenes.",
      worker=dm_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that knows everything about monsters and how to balance fights.",
      worker=bm_agent,
    )
    task = Task(content=prompt)
    response = workforce.process_task(task)
    return response.msg.content
# write a scene
def write_scene(current_scene, next_scene, previous_scene) -> str:
    previous_prompt = "The previous scene contains information and cues that leads to the current scene. Incoorporate these as needed and resolve them. Here is the previous scene: {{" + (previous_scene or "") + "}}. "
    next_prompt = "The scene description should contain information that leads to the next scene. Here is the next scene: {{" + (next_scene or "") + "}}. " if next_scene is not None else ""
    prompt = ("Create a scene description. "
              "Here is the prompt for the current scene you should write a description about: {{" + current_scene + "}}"
              + ( previous_prompt if previous_scene is not None else "")
              + next_prompt  +
              "Do not write a description for the next scene or the previous scene."
              "Don't forget you are writing a description for the current scene."
    )
    scene_response_content = workforce_write_scene(prompt)
    # scene_response_content = single_agent_write_scene(prompt)
    
    print(scene_response_content + "/n")
    
    return scene_response_content

cnt = 0
previous_scene = None
next_scene = None
scene_list = []
for cnt in range(0, len(all_steps) - 1):
    current_scene = all_steps[cnt]
    if cnt != len(all_steps) - 1: 
        next_scene = all_steps[cnt + 1]
        
    current_scene_desc = write_scene(current_scene, next_scene, previous_scene)
    previous_scene = current_scene_desc
    scene_list.append(current_scene_desc)
    cnt += 1
    
